# Nest

Prepares data for the Nest page 

In [1]:
import os
from events import (
    load_event_data, filter_public_events, filter_by_project, prepare, save_files, count_wards
)

Set output directory

In [2]:
OUT_DIR = '../../../../docs/metrics/events/nest/_data/events'

Read in event data and filter by **My LEEDS 2023** project.

The criteria are:

* project type is 'Produce (Signature)'
* event_type contains 'Public Event -'

It's then filtered by project_name is '10 - Melt'

In [3]:
data = (
  load_event_data()
    .pipe(filter_by_project, '10 - Melt')
    .pipe(filter_public_events)
  )
data

,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,ticket_tailor_id,ward_(from_venue)
1098,1338.0,NaN,241,2023-09-09,Melt (Run),['Public Event - Performance'],10 - Melt: Melt (Run),1542.0,['LS26 8AL'],10 - Melt,Produce (Signature),['Season 3 - Dreaming'],2023-09-05,"ev_2809632, ev_2809633, ev_2809634, ev_28096...",['Garforth and Swillington']


Run the processing and save the files

In [4]:
data = data.pipe(prepare).pipe(save_files, OUT_DIR)